In [ ]:
from influxdb_client import InfluxDBClient
import pandas as pd

# Configuration
influxdb_url = "http://localhost:8086"
token = "random_token"
org = "ORG"
bucket = "system_state"
labels = "labels_CC2"
bucket_ano = "anomalies"
dataset = "repad2-cc2-result-minmax"
detections = "repad2-cc2-detection-robust"
margin = 3  # Margin of T values

# Initialize client
client = InfluxDBClient(url=influxdb_url, token=token, org=org)
query_api = client.query_api()

start_time = "1997-04-10T00:00:00Z"

# Construct the Flux query
query_labels = f'''
from(bucket: "{bucket}")
    |> range(start: time(v: "{start_time}"))
    |> filter(fn: (r) => r["_measurement"] == "{labels}")
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
'''

# Construct the Flux query
query_dataset = f'''
from(bucket: "{bucket_ano}")
    |> range(start: time(v: "{start_time}"))
    |> filter(fn: (r) => r["_measurement"] == "{dataset}")
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
'''

# Construct the Flux query
query_detections = f'''
from(bucket: "{bucket_ano}")
    |> range(start: time(v: "{start_time}"))
    |> filter(fn: (r) => r["_measurement"] == "{detections}")
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
'''

# Fetch data
result_dataset = query_api.query_data_frame(query=query_dataset)
result_labels = query_api.query_data_frame(query=query_labels)
result_detections = query_api.query_data_frame(query=query_detections)

# Check if any dataset is empty
if result_dataset.empty or result_detections.empty or result_labels.empty:
    print("One or more datasets are empty. Check data and queries.")
else:
    # Prepare datasets
    result_dataset['timestamp'] = pd.to_datetime(result_dataset['_time'])
    result_labels['timestamp'] = pd.to_datetime(result_labels['_time'])
    result_detections['timestamp'] = pd.to_datetime(result_detections['_time'])

    # Merge to align T values with labels
    full_labels = pd.merge_asof(result_labels.sort_values('timestamp'), result_dataset[['timestamp', 'T']].sort_values('timestamp'), on='timestamp', direction='nearest')

    # Track detection matches
    detection_matches = {index: False for index in result_detections.index}
    label_matches = {index: False for index in full_labels.index}

    # Identify true positives and update match status
    for index, label_row in full_labels.iterrows():
        # Find detections within the margin
        detections_in_margin = result_detections[
            (result_detections['T'] >= label_row['T'] - margin) & 
            (result_detections['T'] <= label_row['T'] + margin)
        ]
        if not detections_in_margin.empty:
            label_matches[index] = True
            for det_index in detections_in_margin.index:
                detection_matches[det_index] = True

    # Calculate true positives, false positives, and false negatives
    true_positives  = sum(match for match in label_matches.values())
    false_negatives = sum(not match for match in label_matches.values())
    false_positives = sum(not match for match in detection_matches.values())

    # Calculating Precision, Recall, and F1 Score
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1_score = 2 * (precision * recall) / (precision + recall)

    # For datasets with no anomalies
    if true_positives + false_negatives + false_negatives  == 0:
        precision = 1
        recall = 1
        f1_score = 1
        print("No anomalies in dataset, and no detections made")

    print(f"True Positives: {true_positives}, False Positives: {false_positives}, False Negatives: {false_negatives}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1_score:.4f}")

    with open(f'../{detections}.txt', 'w') as f:
        print(f"True Positives: {true_positives}, False Positives: {false_positives}, False Negatives: {false_negatives}", file=f)
        print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1_score:.4f}", file=f)
